<a href="https://colab.research.google.com/github/stas420/machine_learning_ev_ice/blob/main/EV_to_ICE_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TODO:** poprawić to, żeby robiło *git push* albo *git clone link* w zależności od potrzeby

In [ ]:
!git clone https://github.com/stas420/machine_learning_ev_ice.git
%cd machine_learning_ev_ice

In [ ]:
# Setup dependencies for the project
%pip install -r requirements.txt

In [ ]:
# Launch labelImg
!labelImg

 Volume in drive E is Lexar NM620 SSD
 Volume Serial Number is 9AF2-AD84

 Directory of e:\studia\machine_learning_ev_ice\src

07/04/2025  21:44    <DIR>          .
07/04/2025  21:44    <DIR>          ..
07/04/2025  21:08             1,632 download_videos.py
07/04/2025  21:27             4,300 EV_to_ICE_ML.ipynb
07/04/2025  21:08                64 requirements.txt
               3 File(s)          5,996 bytes
               2 Dir(s)  102,966,366,208 bytes free


Dżem dobry
:)